# 🧬 FHIR AllergyIntolerance Documentation — Advanced Notebook

This notebook demonstrates a **complete FHIR AllergyIntolerance workflow**, including:

✔ Loading clinical allergy data (from CSV)
✔ Creating valid FHIR AllergyIntolerance resources using `fhir.resources`
✔ SNOMED-coded substances
✔ Reaction + severity modeling
✔ Validation with Pydantic/FHIR schemas
✔ Building a FHIR Bundle
✔ Visualizing severity and allergen distributions

**Everything here is from a coursework project — NOT employer data.**


## 🔧 Step 1 — Import Required Libraries

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from fhir.resources.allergyintolerance import AllergyIntolerance
from fhir.resources.patient import Patient
from fhir.resources.codeableconcept import CodeableConcept
from fhir.resources.coding import Coding
from fhir.resources.fhirdate import FHIRDate
from fhir.resources.bundle import Bundle
from datetime import datetime
import json

sns.set(style="whitegrid")

## 📂 Step 2 — Load Allergy Dataset (30 patients)

In [ ]:
df = pd.read_csv('../data/allergy_example.csv')
df.head()

## 📊 Step 3 — Visualize Allergy Severity Distribution

In [ ]:
plt.figure(figsize=(7,5))
sns.countplot(data=df, x='severity', palette='viridis')
plt.title('Allergy Severity Distribution')
plt.xlabel('Severity')
plt.ylabel('Count')
plt.show()

## 📊 Step 4 — Most Common Allergens

In [ ]:
plt.figure(figsize=(10,6))
sns.countplot(data=df, y='substance', order=df['substance'].value_counts().index, palette='magma')
plt.title('Most Common Allergens')
plt.xlabel('Count')
plt.ylabel('Allergen')
plt.show()

## 🧠 Step 5 — Function to Create FHIR AllergyIntolerance Resource

In [ ]:
def create_allergy_resource(row):
    """
    Converts a single row of allergy data into a FHIR AllergyIntolerance resource.
    """

    allergy = AllergyIntolerance(
        patient=Patient(id=str(row['patient_id'])),
        code=CodeableConcept(
            coding=[Coding(
                system="http://snomed.info/sct",
                code=str(row['substance_code']),
                display=row['substance']
            )]
        ),
        clinicalStatus=CodeableConcept(
            coding=[Coding(
                system="http://terminology.hl7.org/CodeSystem/allergyintolerance-clinical",
                code="active"
            )]
        ),
        verificationStatus=CodeableConcept(
            coding=[Coding(
                system="http://terminology.hl7.org/CodeSystem/allergyintolerance-verification",
                code="confirmed"
            )]
        ),
        reaction=[{
            "description": row['reaction'],
            "severity": row['severity']
        }],
        recordedDate=FHIRDate(row['recorded_date'])
    )

    return allergy

## 🏗 Step 6 — Generate FHIR Resources For All Patients

In [ ]:
resources = [create_allergy_resource(row) for _, row in df.iterrows()]
len(resources)

## 🔍 Step 7 — Example FHIR AllergyIntolerance JSON Output

In [ ]:
example_json = resources[0].json(indent=2)
print(example_json)

## 📦 Step 8 — Build FHIR Bundle With All Allergy Resources

In [ ]:
bundle = Bundle(
    type="collection",
    entry=[{"resource": r.dict()} for r in resources]
)

print(bundle.json(indent=2)[:1000], "\n... (truncated) ...")

## 💾 Step 9 — Save FHIR Bundle to JSON File

In [ ]:
with open('../data/fhir_allergy_bundle.json', 'w') as f:
    f.write(bundle.json(indent=2))

print("FHIR bundle saved to data/fhir_allergy_bundle.json")

# 🎉 Completed!

This notebook demonstrates:

✔ FHIR AllergyIntolerance modeling
✔ Clinical SNOMED coding
✔ Reaction + severity modeling
✔ FHIR Bundle creation
✔ Visualizations of allergy trends
✔ End-to-end documentation workflow

You now have a fully professional FHIR allergy documentation project suitable for your GitHub portfolio.
